In [5]:
import requests
from bs4 import BeautifulSoup

import openmeteo_requests

import requests_cache
from retry_requests import retry

import pandas as pd

import re

import os.path

## Label Collection


In [7]:
def web_parser():
  url = "https://en.wikipedia.org/wiki/Dom_P%C3%A9rignon"
  response = requests.get(url)
  soup = BeautifulSoup(response.text, "html.parser")
  paragraph = soup.find_all("p")[9].text
  vintage_years = {year: "Vintage" for year in list(set(re.findall("\d{4}", paragraph))) if int(year) >= 1940}

  non_vintage_years = {year: "No Vintage" for year in range(1940, 2024) if str(year) not in vintage_years.keys()}

  vintage_df = pd.DataFrame(vintage_years.items(), columns = ["year", "Status"])
  non_vintage_df = pd.DataFrame(non_vintage_years.items(), columns = ["year", "Status"])

  dom_perignon_df = pd.concat([vintage_df, non_vintage_df], axis=0 )
  dom_perignon_df["year"] = dom_perignon_df["year"].astype(str)
  dom_perignon_df.sort_values("year", inplace = True)
  dom_perignon_df.reset_index(inplace = True)
  dom_perignon_df.drop("index", axis = 1, inplace = True)

  dom_perignon_df.to_csv(os.path.join("data","dom_perignon.csv"), index = False)

  return dom_perignon_df

############ Function Call ############
dom_perignon_df = web_parser()
dom_perignon_df

,year,Status
0,1940,No Vintage
1,1941,No Vintage
2,1942,No Vintage
3,1943,Vintage
4,1944,No Vintage
...,...,...
79,2019,No Vintage
80,2020,No Vintage
81,2021,No Vintage
82,2022,No Vintage


## Feature Collection

In [8]:
def api_parser():
  cache_session = requests_cache.CachedSession('.cache', expire_after = -1)
  retry_session = retry(cache_session, retries = 5, backoff_factor = 0.2)
  openmeteo = openmeteo_requests.Client(session = retry_session)

  # Make sure all required weather variables are listed here
  # The order of variables in hourly or daily is important to assign them correctly below
  url = "https://archive-api.open-meteo.com/v1/archive"
  params = {
      "latitude": 49.04,
      "longitude": 3.95,
      "start_date": "1940-01-01",
      "end_date": "2023-12-31",
      "daily": ["weather_code", "temperature_2m_max", "temperature_2m_min", "temperature_2m_mean", "precipitation_sum", "rain_sum", "wind_speed_10m_max", "shortwave_radiation_sum"]
  }
  responses = openmeteo.weather_api(url, params=params)

  # Process first location. Add a for-loop for multiple locations or weather models
  response = responses[0]
  print(f"Coordinates {response.Latitude()}°N {response.Longitude()}°E")
  print(f"Elevation {response.Elevation()} m asl")
  print(f"Timezone {response.Timezone()} {response.TimezoneAbbreviation()}")
  print(f"Timezone difference to GMT+0 {response.UtcOffsetSeconds()} s")

  # Process daily data. The order of variables needs to be the same as requested.
  daily = response.Daily()
  daily_weather_code = daily.Variables(0).ValuesAsNumpy()
  daily_temperature_2m_max = daily.Variables(1).ValuesAsNumpy()
  daily_temperature_2m_min = daily.Variables(2).ValuesAsNumpy()
  daily_temperature_2m_mean = daily.Variables(3).ValuesAsNumpy()
  daily_precipitation_sum = daily.Variables(4).ValuesAsNumpy()
  daily_rain_sum = daily.Variables(5).ValuesAsNumpy()
  daily_wind_speed_10m_max = daily.Variables(6).ValuesAsNumpy()
  daily_shortwave_radiation_sum = daily.Variables(7).ValuesAsNumpy()

  daily_data = {"date": pd.date_range(
      start = pd.to_datetime(daily.Time(), unit = "s", utc = True),
      end = pd.to_datetime(daily.TimeEnd(), unit = "s", utc = True),
      freq = pd.Timedelta(seconds = daily.Interval()),
      inclusive = "left"
  )}
  daily_data["weather_code"] = daily_weather_code
  daily_data["temperature_2m_max"] = daily_temperature_2m_max
  daily_data["temperature_2m_min"] = daily_temperature_2m_min
  daily_data["temperature_2m_mean"] = daily_temperature_2m_mean
  daily_data["precipitation_sum"] = daily_precipitation_sum
  daily_data["rain_sum"] = daily_rain_sum
  daily_data["wind_speed_10m_max"] = daily_wind_speed_10m_max
  daily_data["shortwave_radiation_sum"] = daily_shortwave_radiation_sum

  daily_dataframe = pd.DataFrame(data = daily_data)

  daily_dataframe.to_csv(os.path.join("data","weather_epernay.csv"), index = False)

  return daily_dataframe

############ Function Call ############
raw_weather = api_parser()

Coordinates 49.033390045166016°N 3.9863715171813965°E
Elevation 82.0 m asl
Timezone None None
Timezone difference to GMT+0 0 s


In [48]:
raw_weather

,date,weather_code,temperature_2m_max,temperature_2m_min,temperature_2m_mean,precipitation_sum,rain_sum,wind_speed_10m_max,shortwave_radiation_sum
0,1940-01-01 00:00:00+00:00,3.0,1.792000,-3.9580,-1.178833,NaN,NaN,18.733839,NaN
1,1940-01-02 00:00:00+00:00,0.0,-2.908000,-7.2580,-5.001750,0.0,0.0,18.391735,5.33
2,1940-01-03 00:00:00+00:00,1.0,4.392000,-4.2580,-0.487167,0.0,0.0,16.087610,4.91
3,1940-01-04 00:00:00+00:00,51.0,4.892000,-1.0080,1.800333,0.2,0.2,14.917212,3.34
4,1940-01-05 00:00:00+00:00,51.0,3.342000,-1.5580,0.821167,0.3,0.3,9.746631,3.24
...,...,...,...,...,...,...,...,...,...
30676,2023-12-27 00:00:00+00:00,53.0,10.115500,7.2155,8.609250,1.3,1.3,25.455843,3.73
30677,2023-12-28 00:00:00+00:00,51.0,11.165501,8.3155,9.892583,0.2,0.2,26.230639,2.81
30678,2023-12-29 00:00:00+00:00,51.0,10.565500,8.2655,9.480084,0.6,0.6,31.470673,1.61
30679,2023-12-30 00:00:00+00:00,51.0,10.265500,6.6155,8.663416,0.3,0.3,19.826164,1.62


In [49]:
raw_weather["date"] = raw_weather["date"].astype(str).str.split(expand = True)[0]
raw_weather = raw_weather[ ~raw_weather["date"].str.contains("02-29", na = False)]
raw_weather["year"] = raw_weather["date"].str.split("-", expand=True)[0]
raw_weather["date"] = raw_weather["date"].str.split("-", expand=True)[1] + "-" + raw_weather["date"].str.split("-", expand=True)[2]
raw_weather

/var/folders/sd/r2h508950gv1__s3r96nggxw0000gn/T/ipykernel_62817/3893615434.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  raw_weather["year"] = raw_weather["date"].str.split("-", expand=True)[0]
/var/folders/sd/r2h508950gv1__s3r96nggxw0000gn/T/ipykernel_62817/3893615434.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  raw_weather["date"] = raw_weather["date"].str.split("-", expand=True)[1] + "-" + raw_weather["date"].str.split("-", expand=True)[2]


,date,weather_code,temperature_2m_max,temperature_2m_min,temperature_2m_mean,precipitation_sum,rain_sum,wind_speed_10m_max,shortwave_radiation_sum,year
0,01-01,3.0,1.792000,-3.9580,-1.178833,NaN,NaN,18.733839,NaN,1940
1,01-02,0.0,-2.908000,-7.2580,-5.001750,0.0,0.0,18.391735,5.33,1940
2,01-03,1.0,4.392000,-4.2580,-0.487167,0.0,0.0,16.087610,4.91,1940
3,01-04,51.0,4.892000,-1.0080,1.800333,0.2,0.2,14.917212,3.34,1940
4,01-05,51.0,3.342000,-1.5580,0.821167,0.3,0.3,9.746631,3.24,1940
...,...,...,...,...,...,...,...,...,...,...
30676,12-27,53.0,10.115500,7.2155,8.609250,1.3,1.3,25.455843,3.73,2023
30677,12-28,51.0,11.165501,8.3155,9.892583,0.2,0.2,26.230639,2.81,2023
30678,12-29,51.0,10.565500,8.2655,9.480084,0.6,0.6,31.470673,1.61,2023
30679,12-30,51.0,10.265500,6.6155,8.663416,0.3,0.3,19.826164,1.62,2023


In [50]:
cleaned_data = raw_weather[raw_weather["year"] == "1940"]
cleaned_data.drop(["year"], axis = 1, inplace = True)
cleaned_data.set_index("date", inplace = True)
cleaned_data = cleaned_data.T
cleaned_data.reset_index(inplace = True)
cleaned_data["index"] = cleaned_data["index"] + "Q_1940"
for year in range(1941, 2024):
    data = raw_weather[raw_weather["year"] == str(year)]
    data.drop(["year"], axis = 1, inplace = True)
    data.set_index("date", inplace = True)
    data = data.T
    data.reset_index(inplace = True)
    data["index"] = data["index"] + f"Q_{str(year)}"
    cleaned_data = pd.concat([cleaned_data, data])

cleaned_data


/var/folders/sd/r2h508950gv1__s3r96nggxw0000gn/T/ipykernel_62817/1352339102.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cleaned_data.drop(["year"], axis = 1, inplace = True)
/var/folders/sd/r2h508950gv1__s3r96nggxw0000gn/T/ipykernel_62817/1352339102.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.drop(["year"], axis = 1, inplace = True)
/var/folders/sd/r2h508950gv1__s3r96nggxw0000gn/T/ipykernel_62817/1352339102.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/in

date,index,01-01,01-02,01-03,01-04,01-05,01-06,01-07,01-08,01-09,...,12-22,12-23,12-24,12-25,12-26,12-27,12-28,12-29,12-30,12-31
0,weather_codeQ_1940,3.000000,0.000000,1.000000,51.000000,51.000000,3.000000,53.000000,53.000000,51.000000,...,0.000000,71.000000,2.000000,3.000000,3.000000,73.000000,71.000000,71.000000,73.000000,61.000000
1,temperature_2m_maxQ_1940,1.792000,-2.908000,4.392000,4.892000,3.342000,3.592000,2.942000,4.542000,2.042000,...,-3.408000,-4.058000,-4.408000,-1.258000,-0.408000,3.542000,2.842000,3.942000,7.392000,8.042000
2,temperature_2m_minQ_1940,-3.958000,-7.258000,-4.258000,-1.008000,-1.558000,-0.458000,0.042000,0.442000,-4.008000,...,-7.708000,-8.207999,-7.858000,-7.708000,-4.408000,-0.958000,0.342000,0.492000,1.092000,4.792000
3,temperature_2m_meanQ_1940,-1.178833,-5.001750,-0.487167,1.800333,0.821167,1.502417,1.481583,1.779500,0.058667,...,-5.799666,-6.101751,-6.430917,-4.447584,-2.018417,2.156583,1.831583,2.494083,3.250333,6.396167
4,precipitation_sumQ_1940,NaN,0.000000,0.000000,0.200000,0.300000,0.000000,4.000000,3.400000,0.200000,...,0.000000,0.300000,0.000000,0.000000,0.000000,1.600000,0.300000,0.500000,5.900000,13.300001
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3,temperature_2m_meanQ_2023,13.328000,9.978000,7.090500,10.692584,11.598832,10.492584,9.132167,8.998833,6.371750,...,9.138417,8.892583,9.563416,10.496750,9.744668,8.609250,9.892583,9.480084,8.663416,8.611333
4,precipitation_sumQ_2023,1.600000,7.400000,0.500000,4.299999,3.400000,0.200000,1.500000,14.800001,0.300000,...,2.100000,0.200000,3.000000,1.000000,0.600000,1.300000,0.200000,0.600000,0.300000,2.800000
5,rain_sumQ_2023,1.600000,7.400000,0.500000,4.299999,3.400000,0.200000,1.500000,14.800001,0.300000,...,2.100000,0.200000,3.000000,1.000000,0.600000,1.300000,0.200000,0.600000,0.300000,2.800000
6,wind_speed_10m_maxQ_2023,26.260265,18.681883,22.380884,29.019661,20.532627,23.090569,24.821732,25.117580,27.217524,...,28.241076,24.798223,29.957968,26.758923,21.817902,25.455843,26.230639,31.470673,19.826164,33.605572


In [51]:
weather_code_df = cleaned_data[cleaned_data["index"].str.contains("weather_code")]
weather_code_df["year"] = weather_code_df["index"].str.split("Q_", expand = True)[1]
weather_code_df.drop("index", axis = 1, inplace = True)
weather_code_df.set_index("year", inplace = True)
weather_code_df = weather_code_df.merge(dom_perignon_df, how = "inner", on = "year")
weather_code_df.set_index("year", inplace = True)

temperature_2m_max_df = cleaned_data[cleaned_data["index"].str.contains("temperature_2m_max")]
temperature_2m_max_df["year"] = temperature_2m_max_df["index"].str.split("Q_", expand = True)[1]
temperature_2m_max_df.drop("index", axis = 1, inplace = True)
temperature_2m_max_df.set_index("year", inplace = True)
temperature_2m_max_df = temperature_2m_max_df.merge(dom_perignon_df, how = "inner", on = "year")
temperature_2m_max_df.set_index("year", inplace = True)

temperature_2m_min_df = cleaned_data[cleaned_data["index"].str.contains("temperature_2m_min")]
temperature_2m_min_df["year"] = temperature_2m_min_df["index"].str.split("Q_", expand = True)[1]
temperature_2m_min_df.drop("index", axis = 1, inplace = True)
temperature_2m_min_df.set_index("year", inplace = True)
temperature_2m_min_df = temperature_2m_min_df.merge(dom_perignon_df, how = "inner", on = "year")
temperature_2m_min_df.set_index("year", inplace = True)

temperature_2m_mean_df = cleaned_data[cleaned_data["index"].str.contains("temperature_2m_mean")]
temperature_2m_mean_df["year"] = temperature_2m_mean_df["index"].str.split("Q_", expand = True)[1]
temperature_2m_mean_df.drop("index", axis = 1, inplace = True)
temperature_2m_mean_df.set_index("year", inplace = True)
temperature_2m_mean_df = temperature_2m_mean_df.merge(dom_perignon_df, how = "inner", on = "year")
temperature_2m_mean_df.set_index("year", inplace = True)

precipitation_sum_df = cleaned_data[cleaned_data["index"].str.contains("precipitation_sum")]
precipitation_sum_df["year"] = precipitation_sum_df["index"].str.split("Q_", expand = True)[1]
precipitation_sum_df.drop("index", axis = 1, inplace = True)
precipitation_sum_df.set_index("year", inplace = True)
precipitation_sum_df = precipitation_sum_df.merge(dom_perignon_df, how = "inner", on = "year")
precipitation_sum_df.set_index("year", inplace = True)

rain_sum_df = cleaned_data[cleaned_data["index"].str.contains("rain_sum")]
rain_sum_df["year"] = rain_sum_df["index"].str.split("Q_", expand = True)[1]
rain_sum_df.drop("index", axis = 1, inplace = True)
rain_sum_df.set_index("year", inplace = True)
rain_sum_df = rain_sum_df.merge(dom_perignon_df, how = "inner", on = "year")
rain_sum_df.set_index("year", inplace = True)

wind_speed_df = cleaned_data[cleaned_data["index"].str.contains("wind_speed")]
wind_speed_df["year"] = wind_speed_df["index"].str.split("Q_", expand = True)[1]
wind_speed_df.drop("index", axis = 1, inplace = True)
wind_speed_df.set_index("year", inplace = True)
wind_speed_df = wind_speed_df.merge(dom_perignon_df, how = "inner", on = "year")
wind_speed_df.set_index("year", inplace = True)

shortwave_radiation_df = cleaned_data[cleaned_data["index"].str.contains("shortwave_radiation")]
shortwave_radiation_df["year"] = shortwave_radiation_df["index"].str.split("Q_", expand = True)[1]
shortwave_radiation_df.drop("index", axis = 1, inplace = True)
shortwave_radiation_df.set_index("year", inplace = True)
shortwave_radiation_df = shortwave_radiation_df.merge(dom_perignon_df, how = "inner", on = "year")
shortwave_radiation_df.set_index("year", inplace = True)


/var/folders/sd/r2h508950gv1__s3r96nggxw0000gn/T/ipykernel_62817/1491659473.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  weather_code_df["year"] = weather_code_df["index"].str.split("Q_", expand = True)[1]
/var/folders/sd/r2h508950gv1__s3r96nggxw0000gn/T/ipykernel_62817/1491659473.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  weather_code_df.drop("index", axis = 1, inplace = True)
/var/folders/sd/r2h508950gv1__s3r96nggxw0000gn/T/ipykernel_62817/1491659473.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

In [52]:
with pd.ExcelWriter(os.path.join("data","cleaned_data.xlsx")) as writer:
    weather_code_df.to_excel(writer, sheet_name="weather_code")
    temperature_2m_max_df.to_excel(writer, sheet_name="temperature_2m_max")
    temperature_2m_min_df.to_excel(writer, sheet_name="temperature_2m_min")
    temperature_2m_mean_df.to_excel(writer, sheet_name="temperature_2m_mean")
    precipitation_sum_df.to_excel(writer, sheet_name="precipitation_sum")
    rain_sum_df.to_excel(writer, sheet_name="rain_sum")
    wind_speed_df.to_excel(writer, sheet_name="wind_speed")
    shortwave_radiation_df.to_excel(writer, sheet_name="shortwave_radiation")
